In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
#from sklearn.metrics import 

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

-----
### Train, valid split

In [2]:
pars_smp_train = np.load('data/pars_smp_train.npy')
y_smp_train = np.load('data/y_smp_train.npy')

In [3]:
pars_smp_train.shape, y_smp_train.shape

((1000000, 15, 1), (1000000, 200, 3))

In [4]:
small_pars_smp_train = pars_smp_train[:100].copy()
small_y_smp_train = y_smp_train[:100].copy()

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(small_pars_smp_train, small_y_smp_train, test_size=0.2, shuffle=False, random_state=178)

In [6]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((80, 15, 1), (80, 200, 3), (20, 15, 1), (20, 200, 3))

In [ ]:
class MacroEconomicModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(MacroEconomicModel, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

# Define model parameters
input_size = 3  # Размерность данных о приросте ВВП, инфляции и процентной ставке
hidden_size = 64  # Размер скрытого состояния RNN
output_size = 15  # Размер параметров модели
num_layers = 2  # Количество слоев LSTM


model = MacroEconomicModel(input_size, hidden_size, output_size, num_layers)

# Loss & optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
# CNN BiLSTM
from model_CNN_BiLSTM import CNN_BiLSTM

model = CNN_BiLSTM()

# Loss & optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

[Conv2d(3, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 0)), Conv2d(3, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 0))]


/home/arsenii/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [8]:
y_train = torch.Tensor(y_train)
X_train = torch.Tensor(X_train)

y_valid = torch.Tensor(y_valid)
X_valid = torch.Tensor(X_valid)

batch_size = 60
train_dataset = TensorDataset(y_train, X_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

valid_dataset = TensorDataset(y_valid, X_valid)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

num_epochs = 10

for epoch in range(num_epochs):
    batch_losses = []
    for batch in train_loader:
        inputs, targets = batch
        optimizer.zero_grad()

        outputs = model(inputs).unsqueeze(2)

        # Loss
        loss = torch.sqrt(criterion(outputs, targets))
        batch_losses.append(loss.detach().numpy())

        loss.backward()
        optimizer.step()

    training_loss = np.mean(batch_losses)
    
    with torch.no_grad():
        batch_val_losses = []
        for batch_val in valid_loader:
            inputs_val, targets_val = batch_val
            
            model.eval()

            outputs_val = model(inputs_val).unsqueeze(2)

            # Loss
            loss_val = torch.sqrt(criterion(outputs_val, targets_val))
            batch_val_losses.append(loss_val)

            validation_loss = np.mean(batch_val_losses)

    print(f'Эпоха [{epoch + 1}/{num_epochs}], Потери train: {training_loss.item()}, Loss valid: {validation_loss.item()}')

AttributeError: 'CNN_BiLSTM' object has no attribute 'embed'

In [ ]:
torch.save(model.state_dict(), 'trained_model_100.pth')

In [ ]:
outputs_val.shape

----
### Prediction

In [ ]:
y_smp_test = np.load('data/test/y_smp_test.npy')
y_test = torch.Tensor(y_smp_test)
y_test.shape

In [ ]:
# Create dataloader
test_dataset = TensorDataset(y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Evalute model on test data
with torch.no_grad():
    all_outputs_test = []
    for batch in test_loader:
        inputs_test = batch[0]
        model.eval()

        outputs_test = model(inputs_test).unsqueeze(2)
        all_outputs_test.append(outputs_test)

    # Concat to common tensor
    final_outputs_test = torch.cat(all_outputs_test, dim=0)

In [ ]:
batch_size = 5  # Window size

# Empty tensor for saving result
result = torch.zeros((final_outputs_test.size(0), final_outputs_test.size(1), 6))

for i in range(0, final_outputs_test.size(0), batch_size):
    batch = final_outputs_test[i:i+batch_size]
    
    # Calculate mean
    batch_mean = torch.mean(batch, dim=0).squeeze(1)

    # Sort batch for calculate quantiles
    sorted_batch, _ = torch.sort(batch, dim=0)

    quantiles = torch.quantile(sorted_batch, torch.Tensor([0.1, 0.25, 0.5, 0.75, 0.9]), dim=0)

    # Save to final tensor
    result[i:i+batch_size, :, 0] = batch_mean
    result[i:i+batch_size, :, 1:6] = quantiles

result.shape